In [1]:
from pwn import *
import random
import re, sys

In [2]:
def power(a, n, p):
    res = 1
    a = a % p 
    while n > 0:
        if n % 2:
            res = (res * a) % p
            n = n - 1
        else:
            a = (a ** 2) % p
            n = n // 2
    return res % p

def isPrime(n, k = 10):
    if n == 2 or n == 3:
        return True
    elif n<5:
        return False
    else:
        for i in range(k):
            a = random.randint(2, n - 2)
            if power(a, n - 1, n) != 1:
                return False
    return True

def extended_euclidean(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = extended_euclidean(b % a, a)
        return (g, x - (b // a) * y, y)

def modinv(a, m):
    g, x, y = extended_euclidean(a, m)
    return x % m
  
# function implementing Chinese remainder theorem
# list m contains all the modulii
# list x contains the remainders of the equations
def crt(m, x):
  
    # We run this loop while the list of
    # remainders has length greater than 1
    while True:
          
        # temp1 will contain the new value 
        # of A. which is calculated according 
        # to the equation m1' * m1 * x0 + m0'
        # * m0 * x1
        temp1 = modinv(m[1],m[0]) * x[0] * m[1] + \
                modinv(m[0],m[1]) * x[1] * m[0]
  
        # temp2 contains the value of the modulus
        # in the new equation, which will be the 
        # product of the modulii of the two
        # equations that we are combining
        temp2 = m[0] * m[1]
  
        # we then remove the first two elements
        # from the list of remainders, and replace
        # it with the remainder value, which will
        # be temp1 % temp2
        x.remove(x[0])
        x.remove(x[0])
        x = [temp1 % temp2] + x 
  
        # we then remove the first two values from
        # the list of modulii as we no longer require
        # them and simply replace them with the new 
        # modulii that  we calculated
        m.remove(m[0])
        m.remove(m[0])
        m = [temp2] + m
  
        # once the list has only one element left,
        # we can break as it will only  contain 
        # the value of our final remainder
        if len(x) == 1:
            break
  
    # returns the remainder of the final equation
    return x[0]

In [3]:
N = 200
M = 100
l = list()

while len(l) < N:
    l += filter(isPrime, range(M - 100, M)) # append prime element of [M - 100, M] to l
    M += 100

print(l)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541, 547, 557, 563, 569, 571, 577, 587, 593, 599, 601, 607, 613, 617, 619, 631, 641, 643, 647, 653, 659, 661, 673, 677, 683, 691, 701, 709, 719, 727, 733, 739, 743, 751, 757, 761, 769, 773, 787, 797, 809, 811, 821, 823, 827, 829, 839, 853, 857, 859, 863, 877, 881, 883, 887, 907, 911, 919, 929, 937, 941, 947, 953, 967, 971, 977, 983, 991, 997, 1009, 1013, 1019, 1021, 1031, 1033, 1039, 1049, 1051, 1061, 1063, 1069, 1087, 1091, 1093, 1097, 1103, 1109, 1117, 1123, 1129, 1151, 1153, 1163, 1171, 1181, 1187, 1193, 1201, 1213, 1217, 12

In [6]:
m = []
x = []
i = 0
while (i < N):
    r = remote('crypto.be.ax', 6000)
    g = int(r.recvline().split()[-1])
    p = int(r.recvline().split()[-1])
    ef = int(r.recvline().split()[-1])
    for j in range(64):
        r.recvuntil(b'> ', drop=True)
        r.sendline(str(l[i]).encode())
        ctr = int(r.recvline())
        base = pow(ctr, l[i], p)
        for rr in range(l[i]):
            if (base*pow(g, rr, p)%p == ef):
                m.append(l[i])
                x.append(rr)
                print(l[i], rr)
                break
        i = i+1
        if (i >= N):
            break

[x] Opening connection to crypto.be.ax on port 6000
[x] Opening connection to crypto.be.ax on port 6000: Trying 35.208.182.172
[+] Opening connection to crypto.be.ax on port 6000: Done
[x] Opening connection to crypto.be.ax on port 6000
[x] Opening connection to crypto.be.ax on port 6000: Trying 35.208.182.172
[+] Opening connection to crypto.be.ax on port 6000: Done
[x] Opening connection to crypto.be.ax on port 6000
[x] Opening connection to crypto.be.ax on port 6000: Trying 35.208.182.172
[+] Opening connection to crypto.be.ax on port 6000: Done
[x] Opening connection to crypto.be.ax on port 6000
[x] Opening connection to crypto.be.ax on port 6000: Trying 35.208.182.172
[+] Opening connection to crypto.be.ax on port 6000: Done


In [10]:
from binascii import unhexlify

def long_to_bytes (val, endianness='big'):
    """
    Use :ref:`string formatting` and :func:`~binascii.unhexlify` to
    convert ``val``, a :func:`long`, to a byte :func:`str`.

    :param long val: The value to pack

    :param str endianness: The endianness of the result. ``'big'`` for
      big-endian, ``'little'`` for little-endian.

    If you want byte- and word-ordering to differ, you're on your own.

    Using :ref:`string formatting` lets us use Python's C innards.
    """

    # one (1) hex digit per four (4) bits
    width = val.bit_length()

    # unhexlify wants an even multiple of eight (8) bits, but we don't
    # want more digits than we need (hence the ternary-ish 'or')
    width += 8 - ((width % 8) or 8)

    # format width specifier: four (4) bits per hex digit
    fmt = '%%0%dx' % (width // 4)

    # prepend zero (0) to the width, to zero-pad the output
    s = unhexlify(fmt % val)

    if endianness == 'little':
        # see http://stackoverflow.com/a/931095/309233
        s = s[::-1]

    return s

In [11]:
long_to_bytes(crt(m, x))

b'corctf{qu4drat1c_r3s1due_0r_n0t_1s_7h3_qu3st1on8852042051e57492}'

In [7]:
print(crt(m, x))

5207851285831991069018664616693415824195562260751582516097806772363284738507980478829791747964096025668757404599258689372930479386523996608148861164794493


In [ ]:
5207851285831991069018664616693415824195562260751582516097806772363284738507980478829791747964096025668757404599258689372930479386523996608148861164794493